Import necessary packages

In [3]:
import tensorflow as tf
from keras.applications import ResNet50
from keras import layers, Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import json
import os

Define path for dataset

In [4]:
# Define the relative paths of the image and annotation folders
train_reid_path = r'dataset\plain_re-ID\atrw_reid_train'
train_anno_reid_path = r'dataset\plain_re-ID\atrw_anno_reid_train\reid_list_train.csv'

test_reid_path = r'dataset\plain_re-ID\atrw_reid_test'
test_anno_reid_path = r'dataset\plain_re-ID\atrw_anno_reid_test\reid_list_test.csv'

# Define the absolute paths of the image and annotation folders
train_reid_path = os.path.abspath(train_reid_path)
train_anno_reid_path = os.path.abspath(train_anno_reid_path)

test_reid_path = os.path.abspath(test_reid_path)
test_anno_reid_path = os.path.abspath(test_anno_reid_path)

Load image dataset with the labels

In [5]:
# Load image file paths and corresponding labels
data = pd.read_csv(train_anno_reid_path, names=['tiger_id', 'image_number'])
X_train = np.array(data['image_number'])
y_train = np.array(data['tiger_id'])

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (1887,)
y_train shape: (1887,)


In [6]:
# Split the dataset into train and valid set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create a mapping dictionary for tiger IDs to integers
tiger_id_to_int = {tiger_id: i for i, tiger_id in enumerate(set(y_train))}
int_to_tiger_id = {i: tiger_id for tiger_id, i in tiger_id_to_int.items()}

# Replace the original tiger IDs with the mapped integer values
y_train = [tiger_id_to_int[tiger_id] for tiger_id in y_train]
y_val = [tiger_id_to_int[tiger_id] for tiger_id in y_val]

In [5]:
# Create a function to load and preprocess an image
def load_and_preprocess_image(file_path, label):
    abs_img_path = tf.strings.join([train_reid_path, file_path], separator=os.path.sep)
    image = tf.io.read_file(abs_img_path)  
    image = tf.image.decode_jpeg(image, channels=3) 
    image = tf.image.resize(image, [256, 128])  
    image = tf.cast(image, tf.float32) / 255.0  # Normalization
    return image, label

# Create a tf.data.Dataset for training data
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(load_and_preprocess_image)

# Create a tf.data.Dataset for validation data
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(load_and_preprocess_image)

# Shuffle and batch the datasets
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


Create and train the model

In [6]:
# Create the base model 
base_model = ResNet50(input_shape=(256, 128, 3), include_top=False, weights='imagenet')

# Freeze some layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
num_classes = len(set(y_train))
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
output = layers.Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Train the model
num_epochs = 10
batch_size = 32
model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs)

# Save the model
model_directory_path = os.path.abspath(r'model')
model_path = os.path.join(model_directory_path, 'resnet_50_tf.h5')
model.save(model_path)

Epoch 1/10
48/48 [==============================] - 41s 793ms/step - loss: 4.6008 - accuracy: 0.0477 - val_loss: 4.4421 - val_accuracy: 0.0741
Epoch 2/10
48/48 [==============================] - 40s 821ms/step - loss: 4.3262 - accuracy: 0.0630 - val_loss: 4.2515 - val_accuracy: 0.0794
Epoch 3/10
48/48 [==============================] - 40s 822ms/step - loss: 4.1247 - accuracy: 0.0954 - val_loss: 4.1073 - val_accuracy: 0.0899
Epoch 4/10
48/48 [==============================] - 40s 820ms/step - loss: 3.9776 - accuracy: 0.1034 - val_loss: 3.9648 - val_accuracy: 0.1164
Epoch 5/10
48/48 [==============================] - 40s 823ms/step - loss: 3.8595 - accuracy: 0.1140 - val_loss: 3.9057 - val_accuracy: 0.1349
Epoch 6/10
48/48 [==============================] - 41s 833ms/step - loss: 3.7794 - accuracy: 0.1199 - val_loss: 3.8211 - val_accuracy: 0.1138
Epoch 7/10
48/48 [==============================] - 40s 830ms/step - loss: 3.6820 - accuracy: 0.1226 - val_loss: 3.6780 - val_accuracy: 0.1296

c:\Users\22498\anaconda3\envs\amur_tiger_re-identificiation\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Generate predictions

In [6]:
# Load testing data
data = pd.read_csv(train_anno_reid_path, names=['tiger_id', 'image_number'])
X_test = np.array(data['image_number'])
y_test = np.array(data['tiger_id']) 

# Apply the preprocessing function to the testing data
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.map(load_and_preprocess_image)

batch_size = 32
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [7]:
# Load model 
model_directory_path = os.path.abspath(r'model')
model_path = os.path.join(model_directory_path, 'resnet_50_tf.h5')
model = tf.keras.models.load_model(model_path)

query_ids = []
ans_ids_list = []

for images, labels in test_dataset:
    predictions = model.predict(images)
    
    # Get the top 5 predicted labels (tiger IDs) for each image
    top5_indices = np.argsort(predictions, axis=1)[:, -5:]
    
    query_ids.extend(labels.numpy())
    ans_ids_list.extend(top5_indices.tolist())

1/1 [==============================] - 1s 1s/step


Produce JSON file

In [12]:
json_list = []

for query_id, ans_ids in zip(query_ids, ans_ids_list):
    predicted_tiger_ids = [int(int_to_tiger_id[i]) for i in ans_ids]
    json_entry = {"query_id": int(query_id), "ans_ids": predicted_tiger_ids}

    json_list.append(json_entry)

In [13]:
json_list

[{'query_id': 250, 'ans_ids': [250, 33, 237, 153, 160]},
 {'query_id': 256, 'ans_ids': [244, 114, 261, 246, 249]},
 {'query_id': 171, 'ans_ids': [168, 172, 171, 154, 175]},
 {'query_id': 247, 'ans_ids': [250, 55, 237, 160, 153]},
 {'query_id': 238, 'ans_ids': [55, 250, 237, 160, 153]},
 {'query_id': 264, 'ans_ids': [247, 267, 66, 246, 249]},
 {'query_id': 54, 'ans_ids': [261, 244, 154, 246, 249]},
 {'query_id': 237, 'ans_ids': [55, 33, 237, 153, 160]},
 {'query_id': 255, 'ans_ids': [267, 244, 261, 246, 249]},
 {'query_id': 96, 'ans_ids': [55, 33, 237, 153, 160]},
 {'query_id': 114, 'ans_ids': [267, 247, 261, 246, 249]},
 {'query_id': 100, 'ans_ids': [267, 265, 247, 246, 249]},
 {'query_id': 201, 'ans_ids': [196, 168, 226, 201, 198]},
 {'query_id': 91, 'ans_ids': [243, 55, 237, 153, 246]},
 {'query_id': 168, 'ans_ids': [154, 171, 168, 172, 175]},
 {'query_id': 114, 'ans_ids': [261, 115, 246, 114, 249]},
 {'query_id': 153, 'ans_ids': [55, 33, 237, 153, 160]},
 {'query_id': 256, 'ans_ids'

In [14]:
output_path = r'output/predictions.json'
output_path = os.path.abspath(output_path)
with open(output_path, 'w') as json_file:
    json.dump(json_list, json_file)